Идеи:
- Можно использовать SeqToSeq модель, например, T5 с выходом в виде строк вроде "9.3"
- Применить идеалогию изображений: использовать вектора слов как строки или столбцы
- Нужна функция, которая возвращает N матриц X для обучения
- https://docs.fast.ai/tutorial.transformers
https://www.reddit.com/r/MLQuestions/comments/e8kyc3/finetuning_bert_for_a_regression_task/

# Модель оценки заголовка

Введенные или сгенерированные заголовки нужно как-то оценивать. Так как главной целью обычно является увеличение количества просмотров, то в качестве критерия можно использовать число просмотров у ранее опубликованных статей.

Таким образом, перед нами стоит задача регрессии: на входе заголовок, на выходе число (балл от 0 до 10 с точностью 0.1).

В качестве библиотеки, реализующей модель русского языка, мы используем spaCy (в первую очередь из соображений скорости).

Для работы с текстами мы используем библиотеку [Transformers](https://huggingface.co/transformers/), обладающую высокой эффективностью для задач «понимания» текста (NLU) и его генерации (NLG). Библиотека предоставляет удобный интерфес для работы с предобученными NLP-моделями на основе архитектуры transformer.

## 1. Подготовка датасета

В нашем распоряжении множество данных, полученных в результате парсинга. Объединим их в один большой датасет для построения модели оценки заголовков. Для его построения нам нужны те датасеты (сайты), для которых имеется информация о количестве просмотров статьи.

In [2]:
import warnings
warnings.filterwarnings('ignore')

# standard libraries
import pickle
import io

# data processing libraries
import numpy as np
import pandas as pd

# make numpy printouts easier to read
np.set_printoptions(precision=3, suppress=True)

# data processing progress bar
from tqdm.auto import tqdm
tqdm.pandas()

# visualization
import matplotlib.pyplot as plt
import seaborn as sns

# обработка естественного языка
#import spacy
#nlp = spacy.load("ru_core_news_lg")

# библиотеки для машинного и глубокого обучения
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse

import torch
import torch.nn.functional as F

if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu"  
device = torch.device(dev)  

from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
model_name = 'DeepPavlov/rubert-base-cased'

%matplotlib inline

In [3]:
# пути к датасетам
DATASETS_PATH = "/home/leo/DATASETS"
# TOKENIZED_TITLES_PATH = f"{DATASETS_PATH}/tokenized_titles.pickle"

# словарь с источником данных и их характеристиками
with open('../sources.pickle', 'rb') as f:
    sources = pickle.load(f)
    
# Токенизация большого числа заголовков — затратная по времени операция.
# Поэтому предварительно токенизированные заголовки хранятся в виде
# сжатого датафрайма
#tokenized_titles = pd.read_pickle(TOKENIZED_TITLES_PATH, compression='gzip')

In [4]:
# for i in [3, 8, 9]:
#     spacy.displacy.render(tokenized_titles.iloc[i], style='ent', jupyter=True)

In [5]:
# tokenized_titles.to_pickle(path=TOKENIZED_TITLES_PATH, compression='gzip')

In [6]:
# соединяем датасеты в один общий датасет с именем df
dfs = dict()

for source in sources:
    dfs[source] = pd.read_csv(f"{DATASETS_PATH}/{source}.csv",
                              index_col=0,
                              parse_dates=['post_time', 'parse_time'])
    dfs[source]['source'] = source
    
df = pd.concat(dfs[key] for key in dfs)

# удаляем дубликаты
df = df.drop_duplicates()

# преобразуем количество просмотров к текстовому значению
df.views_num = df.views_num.apply(lambda x: int(''.join(filter(str.isdigit, str(x)))))

# удаляем записи без просмотров (обычно это закрытые и недоступные статьи)
df = df.drop(df[df.views_num == 0.0].index)

# объединим датасет с токенизированные заголовки
# df = df.join(tokenized_titles)
df = df.loc[~df.index.duplicated(keep='last')]

# 2. Векторизация текста и построение модели на PyTorch

Начнем с базовой модели: сопоставим каждому заголовку векторное представление в виде набора векторов для слов, дополним векторные представления нулевыми векторами, сложим их в трехмерный тензор, обучим модель для оценки заголовков и проверим качество скользящим контролем.

In [7]:
# В качестве конечных данных нам нужны лишь сведения
# о токенах заголовков и количестве просмотров статей.
Xy = df[['title', 'views_num']]

# удаляем пропущенные значения, если таковые есть
Xy = Xy.dropna(axis=0)

# разобьем все группы на 21 класс от 0 до 20
# по числу оценок от 0.0 до 10.0 с шагом 0.5
Xy = Xy.sort_values(by='views_num')
score = np.linspace(0, 1, Xy.shape[0])
Xy['score'] = score
Xy = Xy.drop(columns='views_num')

# представим данные в виде кортежей (токенизированный текст, метка)
# data = Xy.apply(lambda row: (row['doc'], row['score']), axis=1).to_list()

In [8]:
max_length = Xy.title.apply(len).max()

In [55]:
text2text_generator = pipeline("text2text-generation")

In [61]:
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# в библиотеке transformers таск sentiment-analysis
# соответствует TextClassificationPipeline
# classifier = pipeline(task="sentiment-analysis",
#                       model=model,
#                       tokenizer=tokenizer)

KeyboardInterrupt: 

In [10]:
from transformers import AdamW
optimizer = AdamW(model.parameters(), lr=1e-5)

In [11]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5)

In [44]:
batch_size = 100
text_batch = Xy[:batch_size].title.to_list()
encoding = tokenizer(text_batch,
                     return_tensors='pt',
                     max_length=max_length,
                     padding=True,
                     truncation=True)
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

In [45]:
labels = torch.tensor(Xy[:batch_size].score.apply(lambda x: round(x*10)).to_list())

In [49]:
from torch.nn import functional as F
outputs = model(input_ids, attention_mask=attention_mask)
loss = F.cross_entropy(outputs.logits, labels)
loss.backward()
optimizer.step()

In [48]:
loss

tensor(0.2744, grad_fn=<NllLossBackward>)

In [50]:
loss.backward()
optimizer.step()

RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling .backward() or autograd.grad() the first time.

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total # of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset            # evaluation dataset
)

In [ ]:
max_tokens_in_title = Xy.doc.apply(len).max()
cols_num = 96

def make_stack(doc):
    # представляем заголовок как набор векторов
    words_stack = np.vstack(word.vector for word in doc)
    
    # дополняем плоскость нулями
    zeros_rows_num = max_tokens_in_title-words_stack.shape[0]
    zeros_stack = np.zeros((zeros_rows_num, cols_num))
    plate_stack = np.vstack([words_stack, zeros_stack])
    return plate_stack

In [ ]:
y = Xy.views_num.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)

In [ ]:
X_train = torch.from_numpy(X_train).float().to(device)

In [ ]:
y_train = torch.from_numpy(y_train).float().to(device)

In [ ]:
y_train

# 4. Проверка дополнительных гипотез

In [ ]:
# генерация дополнительных признаков
# Xy.loc[:, ['title']] = Xy.title.apply(str)

# Xy.loc[:, ['doc']] = Xy.title.progress_apply(nlp)

# длина заголовка в символах
# Xy.loc[:, ['len']] = Xy.title.apply(len)

# количество токенов
# Xy.loc[:, ['tokens_num']] = Xy.tokens.apply(lambda x: len(x))